# Amostragem

## Carregamento da base de dados

In [118]:
# Carregamento da base de dados
import pandas as pd
import random
import numpy as np

In [119]:
# Carregamento dos dados do arquivo csv
dataset = pd.read_csv('..\\base-de-dados\\credit_data.csv')

In [120]:
dataset.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [121]:
dataset.tail()

,i#clientid,income,age,loan,c#default
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0
1999,2000,69436.579552,56.152617,7378.833599,0


## Amostragem aleatória simples

In [124]:
def amostragem_aleatoria_simples(dataframe, amostras):
    return dataset.sample(n = amostras)

In [125]:
df_amostra_aleatoria_simples =amostragem_aleatoria_simples(dataset, 1000)
df_amostra_aleatoria_simples.shape

(1000, 5)

## Amostragem Sistemática

In [108]:
# Função
def amostragem_sistematica(dataset, amostras):
    # intervalo de amostragem
    intervalo = len(dataset) // amostras
    # Fixa um número aleatório
    random.seed(1)
    inicio = random.randint(0, intervalo)
    # Retorna os indices escolhidos do dataset
    indices = np.arange(inicio, len(dataset), step=intervalo)
    # Retorna as linhas do dataset em função dos indices selecionados
    amostra_sistematica = dataset.iloc[indices]
    return amostra_sistematica

In [127]:
df_amostra_sistematica = amostragem_sistematica(dataset, 1000)
df_amostra_sistematica.shape

(1000, 5)

## Amostragem por grupos

In [109]:
# Função para resolução do problema
def amostragem_agrupamento(dataset, numero_grupos):
    intervalo = len(dataset) / numero_grupos
    grupos = []
    id_grupo = 0
    contagem = 0
    for _ in dataset.iterrows():
        grupos.append(id_grupo)
        contagem += 1
        if contagem > intervalo:
            contagem = 0
            id_grupo += 1
    dataset['grupo'] = grupos
    # Fixar o random
    random.seed(1)
    grupo_selecionado = random.randint(0, numero_grupos)
    return dataset[dataset['grupo'] == grupo_selecionado]

In [130]:
len(dataset) / 2

1000.0

In [132]:
df_amostra_agrupamento = amostragem_agrupamento(dataset, 2)
df_amostra_agrupamento.shape, df_amostra_agrupamento['grupo'].value_counts()

((1001, 6),
 0    1001
 Name: grupo, dtype: int64)

## Amostra Estratificada

In [133]:
# Aprendizagem de máquina para abstração
from sklearn.model_selection import StratifiedShuffleSplit

In [136]:
dataset['c#default'].value_counts()

0    1717
1     283
Name: c#default, dtype: int64

In [139]:
def amostragem_estratificada(dataset, percentual, campo):
    split = StratifiedShuffleSplit(test_size=percentual, random_state=1)
    for _, y in split.split(dataset, dataset[campo]):
        # amostra 10% para teste
        df_y = dataset.iloc[y]
    return df_y

In [140]:
df_amostra_estratificada = amostragem_estratificada(dataset, 0.5, "c#default")
df_amostra_estratificada.shape

(1000, 6)

## Amostragem de Reservatório

In [142]:
# Função
def amostragem_reservatorio(dataset, amostras):
    stream = []
    for i in range(len(dataset)):
        stream.append(i)
    i = 0
    tamanho = len(dataset)
    reservatorio = [0] * amostras
    for i in range(amostras):
        reservatorio[i] = stream[i]
    while i < tamanho:
        j = random.randrange(i+1)
        if j < amostras:
            reservatorio[j] = stream[i]
        i += 1
    return dataset.iloc[reservatorio]

In [143]:
df_amostragem_reservatorio = amostragem_reservatorio(dataset, 1000)
df_amostra_reservatorio.shape

(100, 5)

## Comparativo dos resultados

In [126]:
media_age = []
media_age.append(dataset['age'].mean())
media_age.append(df_amostra_aleatoria_simples['age'].mean())
media_age.append(df_amostra_sistematica['age'].mean())
media_age.append(df_amostra_agrupamento['age'].mean())
media_age.append(df_amostra_estratificada['age'].mean())
media_age.append(df_amostra_reservatorio['age'].mean())
print(f'Média age{media_age}')
m = media_age[0]
for a in media_age:
    print(f'{a - m:.2f}', end=", ")
print("")


KeyError: ('age', 1000)

In [114]:
media_income = []
media_income.append(dataset['income'].mean())
media_income.append(df_amostra_aleatoria_simples['income'].mean())
media_income.append(df_amostra_sistematica['income'].mean())
media_income.append(df_amostra_agrupamento['income'].mean())
media_income.append(df_amostra_estratificada['income'].mean())
media_income.append(df_amostra_reservatorio['income'].mean())
print(f'Média income{media_income}')
m = media_income[0]
for a in media_income:
    print(f'{a - m:.2f}', end=", ")
print("")

Média income[45331.600017793244, 45518.06857786141, 42839.840865461156, 47605.51129301478, 39516.1268431028, 44835.662659141235]
0.00, 186.47, -2491.76, 2273.91, -5815.47, -495.94, 


In [115]:
media_loan = []
media_loan.append(dataset['loan'].mean())
media_loan.append(df_amostra_aleatoria_simples['loan'].mean())
media_loan.append(df_amostra_sistematica['loan'].mean())
media_loan.append(df_amostra_agrupamento['loan'].mean())
media_loan.append(df_amostra_estratificada['loan'].mean())
media_loan.append(df_amostra_reservatorio['loan'].mean())
print(f'Média loan{media_loan}')
m = media_loan[0]
for a in media_loan:
    print(f'{a - m:.2f}', end=", ")
print("")

Média loan[4444.369694688258, 4560.687626884299, 4742.209210616534, 6087.659724430348, 3913.890880279618, 4560.853171789891]
0.00, 116.32, 297.84, 1643.29, -530.48, 116.48, 
